In [22]:
# Install Dependencies

import numpy as np
import pandas as pd
import pickle
from statistics import mode
import nltk
from nltk import word_tokenize
from nltk.stem import LancasterStemmer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras import models # type: ignore
from tensorflow.keras import backend as K # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore
from tensorflow.keras.preprocessing.text import Tokenizer  # type: ignore
from tensorflow.keras.utils import plot_model # type: ignore
from tensorflow.keras.layers import Input,LSTM,Embedding,Dense,Concatenate,Attention # type: ignore
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup

[nltk_data] Downloading package wordnet to C:\Users\streamteck.STREAMT
[nltk_data]     ECK\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\streamteck.STREA
[nltk_data]     MTECK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\streamteck.STREAMTEC
[nltk_data]     K\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("data/Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [9]:
df.isnull().sum()
# df.nunique()

Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

### Pre-processing

In [11]:
df.dropna(axis=0, inplace=True)
df.drop_duplicates(subset=['Text'], inplace=True)

In [12]:
input_data = df.loc[:, 'Text']
target_data = df.loc[:, 'Summary']
target_data.replace('', np.nan, inplace=True)

In [26]:
input_texts = []
target_texts = []
input_words = []
target_words = []

contractions = pickle.load(open("contractions.pkl", "rb"))['contractions']
stop_words = (stopwords.words('english'))
stemm = LancasterStemmer()

In [27]:
def clean(texts, src):
    texts = BeautifulSoup(texts, "html.parser").text
    words=word_tokenize(texts.lower())
    words=list(filter(lambda w: (w.isalpha() and len(w) >= 3), words))
    words = [contractions(w) if w in contractions else w for w in words]
    if src == "inputs":
        words = [stemm.stem(w) for w in words if w not in stop_words]
    else:
        words = [w for w in words if w not in stop_words]
    return words

In [28]:
for in_text, tg_text in zip(input_data, target_data):
    in_words = clean(in_text, "inputs")
    input_texts += [' '.join(in_words)]
    input_words += in_words

    tg_words = clean("sos " + tg_text + " eos", "target")
    target_texts += [' '.join(tg_words)]
    target_words += tg_words

C:\Users\streamteck.STREAMTECK\AppData\Local\Temp\ipykernel_4220\325766350.py:2: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a filename than HTML or XML.

If you meant to use Beautiful Soup to parse the contents of a file on disk, then something has gone wrong. You should open the file first, using code like this:

    filehandle = open(your filename)

You can then feed the open filehandle into Beautiful Soup instead of using the filename.

However, if you want to parse some data that happens to look like a filename, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  texts = BeautifulSoup(texts, "html.parser").text
